# Assignment 1: Data Extraction

Student: Koray Poyraz <br>
Studentnr: 5367646

# Task 1

link to database: BikeStoresDatabase on Teams

The database represents the relations of a bike store which keeps track of their stores, customers, products etc and manages data such as orders, ordered items.

|.     | Tables |
| -------|:----:|
| amount | 9   |

| Relation name   | tuples | description |
| ------------ |:----:|:----------:|
| stores       | 3    | keeps track of the stores they have |
| staffs       | 10   | keeps track of their staffs, such as name, email, phone number ...    |
| customers    | 13   | keeps track of their customers such as name, phone, email, address ...  |
| brands       | 9    | keeps track of their brands such as name  |
| categories   | 7    | keeps track of the categories of their products  |
| orders       | 1615 | keeps track of the orders of their customers such as order status, date ...   |
| products     | 321  | keeps track of their products such as name, year, brand ...   |
| order_items  | 4722 | keeps track of the ordered items such as quantity, product, order ...      |
| stocks       | 939  | keeps track of their stocks such as quantity of products they have in stock      |

stores(store_id, store_name, )

# TASK 2

## Q1

### NL
Display the first name and last name of all customers and the order id of any orders they might have

### RA
∏ customers.first_name, customers.last_name, o.order_id (customers ⟕ customers.customer_id=o.customer_id Po(orders))

### SQL query
select customers.first_name, customers.last_name, o.order_id from customers left outer join orders o on customers.customer_id = o.customer_id;

# Q2

### NL
Display count of all customers who don't have a phone number

### RA
γ count(customer_id) as n_null_values (σ grade=NULL takes)

### SQL
select count(customer_id) as n_null_values from customers where phone is null;

# Q3

### NL
Display the first and last name of all staffs that has no orders

### RA
∏ a.first_name, a.last_name
 (Pa(staffs) -
  (∏ b.first_name, b.last_name
   (σ b.staff_id=orders.staff_id
    (Pb(staffs) X orders))))

### SQL
select a.first_name, a.last_name from staffs a EXCEPT select b.first_name, b.last_name from staffs b, orders where b.staff_id=orders.staff_id;


# Q4

# NL
Display product names of which only 3 are in stock

### RA
∏ p.product_name, s.quantity (σ s.quantity=3 (Ps(stocks) ⋈ s.product_id=p.product_id Pp(products)))

### SQL
select p.product_name, s.quantity from stocks s inner join products p on s.product_id=p.product_id where s.quantity = 3;

# Q5
### NL
Display category name and the number of products each category has

ps: I decided to only display 5/7 categories by using limit as the question asked for max 5 distinct values.

### RA
c.category_name $\gamma$ COUNT() as n_products (products ⋈ c.category_id = products.category_id Pc(categories))

### SQL
select c.category_name, count() as n_products from products
    inner join categories c on c.category_id = products.category_id
group by c.category_name
limit 5;

# Q6

### NL
Display the records of products that are the max and min prices

### RA
σ list_price=( $\gamma$ MIN(list_price)(products)) OR list_price=( $\gamma$ MAX(list_price)(products)) (products)

### SQL
select * from products where list_price=(select min(list_price) from products) or list_price=(select max(list_price) from products);

# Q7
### NL
Display all staffs with the staff id, staff name and his/ her managers id and managers name

### RA
∏ staff_id, staff_name, manager_id, manager_name <br>
 P(a.staff_id → staff_id, a.first_name → staff_name, b.staff_id → manager_id, b.first_name → manager_name) <br>
  (σ a.manager_id = b.staff_id (Pa(staffs) X (Pb(staffs)))

### SQL
select a.staff_id as staff_id, a.first_name as staff_name, b.staff_id as manager_id, b.first_name as manager_name from staffs a, staffs b where a.manager_id=b.staff_id;

# TASK 3

In [2]:
import sqlite3
import pandas as pd
import numpy as np

In [3]:
# 3.1

def fetch_table_names(db_conn):
    cur = db_conn.cursor()
    cur.execute('select DISTINCT tbl_name from sqlite_master')
    return cur.fetchall()

def fetch_table(db_conn, table):
    header_query = "SELECT name FROM pragma_table_info('" + table + "') ORDER BY cid;"
    cols_init = db_conn.cursor().execute(header_query).fetchall()
    cols = [''.join(i) for i in cols_init]
    result = db_conn.cursor().execute('select * from ' + table).fetchall()
    return pd.DataFrame(result, columns=cols)

In [3]:
conn = sqlite3.connect('../data/BikeStoresDatabase.db')
for table_name in fetch_table_names(conn):
    table = ''.join(table_name)
    fetch_table(conn, table).to_csv('../csv/'+table, index=False)

conn.close()
print('finished gen csv files')

finished gen csv files


In [4]:
# 3.2

def read_csv(table_name):
    return pd.read_csv(f'../csv/{table_name}')

In [5]:
# Q1

# select customers.first_name, customers.last_name, o.order_id from customers left outer join orders o on customers.customer_id = o.customer_id;
df_customers = read_csv('customers')
df_orders = read_csv('orders')

merged = pd.merge(df_customers, df_orders, left_on='customer_id', right_on='customer_id', how='left')
merged[['first_name', 'last_name', 'order_id']]

,first_name,last_name,order_id
0,Debra,Burks,599
1,Debra,Burks,1555
2,Debra,Burks,1613
3,Kasha,Todd,692
4,Kasha,Todd,1084
...,...,...,...
1610,Jamaal,Morrison,1366
1611,Cassie,Cline,1036
1612,Lezlie,Lamb,558
1613,Ivette,Estes,616


In [6]:
# Q2
# select count(*) as n_null_values from customers where phone is null;
n_null_values = len(df_customers[df_customers['phone'].isna()])
print(f'num null values: {n_null_values}')

num null values: 1267


In [7]:
# Q3
# select a.first_name, a.last_name from staffs a EXCEPT select b.first_name, b.last_name from staffs b, orders where b.staff_id=orders.staff_id;
df_staffs = read_csv('staffs')

staffs_with_orders = pd.merge(df_staffs, df_orders, on='staff_id')
has_no_orders = ~df_staffs['staff_id'].isin(staffs_with_orders['staff_id'])
df_staffs[has_no_orders][['first_name', 'last_name']]

,first_name,last_name
0,Fabiola,Jackson
3,Virgie,Wiggins
4,Jannette,David
9,Bernardine,Houston


In [8]:
# Q4
# select p.product_name, stocks.quantity from stocks inner join products p on stocks.product_id=p.product_id where quantity = 3;
df_stocks = read_csv('stocks')
df_products = read_csv('products')

df_stocks = df_stocks[df_stocks['quantity'] == 3]
stocks_x_products = pd.merge(df_stocks, df_products, on='product_id')

# I use here head() function to make the table shorter for better reading
stocks_x_products[['product_name', 'quantity']].head()

,product_name,quantity
0,Electra Moto 1 - 2016,3
1,Electra Amsterdam Original 3i - 2015/2017,3
2,Trek Marlin 6 - 2018,3
3,Trek Remedy 9.8 27.5 - 2018,3
4,Trek Remedy 9.8 27.5 - 2018,3


In [9]:
# Q5
# select c.category_name, count() as n_products from products
#     inner join categories c on c.category_id = products.category_id
# group by c.category_name
# limit 5;

df_products = read_csv('products')
df_categories = read_csv('categories')

pd.merge(df_products, df_categories, on='category_id', how='inner')\
    .groupby('category_name')\
    .size()\
    .reset_index()\
    .rename(columns={0:'n_products'})\
    .head(5)

# Reset_index makes category name a column and gives the possibility to rename count() column to n_products

,category_name,n_products
0,Children Bicycles,59
1,Comfort Bicycles,30
2,Cruisers Bicycles,78
3,Cyclocross Bicycles,10
4,Electric Bikes,24


In [10]:
# Q6
# select * from products where list_price=(select min(list_price) from products) or list_price=(select max(list_price) from products);

df_products[(df_products['list_price'] == df_products['list_price'].max()) | (df_products['list_price'] == df_products['list_price'].min())]

,product_id,product_name,brand_id,category_id,model_year,list_price
154,155,Trek Domane SLR 9 Disc - 2018,9,7,2018,11999.99
262,263,Strider Classic 12 Balance Bike - 2018,6,1,2018,89.99


In [11]:
# Q7
# select a.staff_id as staff_id, a.first_name as staff_name, b.staff_id as manager_id, b.first_name as manager_name from staffs a, staffs b where a.manager_id=b.staff_id;

df_staffs = read_csv('staffs')
self_joined_staffs = pd.merge(df_staffs, df_staffs, left_on='manager_id', right_on='staff_id')

staff_id='staff_id'
staff_name='staff_name'
manager_id='manager_id'
manager_name='manager_name'

renamed_columns=self_joined_staffs.rename(columns={'staff_id_x':staff_id, 'first_name_x':staff_name, 'staff_id_y':manager_id, 'first_name_y':manager_name})
renamed_columns[[staff_id, staff_name, manager_id, manager_name]]

,staff_id,staff_name,manager_id,manager_name
0,2,Mireya,1,Fabiola
1,5,Jannette,1,Fabiola
2,8,Kali,1,Fabiola
3,3,Genna,2,Mireya
4,4,Virgie,2,Mireya
5,6,Marcelene,5,Jannette
6,7,Venita,5,Jannette
7,9,Layla,7,Venita
8,10,Bernardine,7,Venita


# Task 4

# Q1
### A
Definition of division in relational algebra:

The division operator of relational algebra, “÷”, is defined as follows. Let r(R) and s(S) be relations, and let S ⊆ R; that is, every attribute of schema S is also in schema R.
Given a tuple t, let t[S] denote the projection of tuple t on the attributes in S. Then r ÷ s is a relation on schema R − S (that is, on the schema containing all attributes of schema R that are not in schema S).
A tuple t is in r ÷ s if and only if both of two conditions hold:
- t is in ΠR−S(r)
- For every tuple ts in s, there is a tuple tr in r satisfying both of the following:
    - a. tr[S] = ts[S]
    - b. tr[R − S] = t

Own words:

Division operator R ÷ S can be applied if and only if:
- attributes of S is proper subset of attributes of R
- the relation returned by division operator will have attributes = (all attributes of R - all attributes of S)
- the relation returned by division operator will return those tuples from relation A which are associated to every S's tuple

The relation returned by division operator will return those tuples from relation R which are associated to every S’s tuple.

E.g. display all customers (R) id who have account <u>at all</u> branches (S) in city Amsterdam.

### B
R ÷ S is defined as: ∏R-S(r) − ∏R-S((∏R-S(r) x s) − ∏R-S,S(r))

In our example of customers branches in city Amsterdam we can say the following:

- r <- ∏ customers.customerId, account.bname (customers ⋈ customers.customerId=account.customerId account)
- s <- ∏ bname (σ city='Amsterdam'(branches))

- ∏R-S(r) is here ∏ customers.customerId (r) (no duplicate elements)
- ∏R-S,S(r) is here ∏ customers.customerId, account.bname (r) = (The projection to get the fields in the right order for substraction)
- ∏R-S(r) x s is here ∏ customers.customerId (r) x {bname 1, bname 2, ...}, all possible combinations on branches in city Amsterdam

# Q2
(X ^ Y) v (~X ^ ~Y)

# Q3

<p>(𝐴∨𝐵) ∧ (~𝐴 ∨ 𝐶) ∧ (𝐵 ∨ 𝐶) = (𝐴 ∨ 𝐵) ∧ (~𝐴 ∨ 𝐶)</p>

| A	| B | C | (A ∨ B) | (¬A ∨ C)| (B ∨ C) | LHS | RHS |
|---|:-:|:---:| :---:|:---:|:---:|:---:|:---:|
| 0	| 0 | 0 | 0 | 1 | 0 | 0 | 0 |
| 0	| 0 | 1 | 0 | 1 | 1 | 0 | 0 |
| 0	| 1 | 0 | 1 | 1 | 1 | 1 | 1 |
| 0	| 1 | 1 | 1 | 1 | 1 | 1 | 1 |
| 1	| 0 | 0 | 1 | 0 | 0 | 0 | 0 |
| 1	| 0 | 1 | 1 | 1 | 1 | 1 | 1 |
| 1	| 1 | 0 | 1 | 0 | 1 | 0 | 0 |
| 1	| 1 | 1 | 1 | 1 | 1 | 1 | 1 |

LHS=RHS